In [24]:
import random
import csv
import itertools

In [25]:
# -------------------------------
# Cache Line and Cache Simulator
# -------------------------------
class CacheLine:
    def __init__(self, tag):
        self.tag = tag
        self.timestamp = 0

class CacheSimulator:
    def __init__(self, size, block_size, associativity, replacement_policy):
        self.size = size
        self.block_size = block_size
        self.associativity = associativity
        self.replacement_policy = replacement_policy
        self.num_sets = self.size // (self.block_size * self.associativity)
        self.cache = {i: [] for i in range(self.num_sets)}
        self.hits = 0
        self.misses = 0
        self.time = 0

    def get_set_and_tag(self, address):
        block_addr = address // self.block_size
        set_index = block_addr % self.num_sets
        tag = block_addr // self.num_sets
        return set_index, tag

    def access(self, address):
        set_index, tag = self.get_set_and_tag(address)
        self.time += 1
        lines = self.cache[set_index]

        for line in lines:
            if line.tag == tag:
                self.hits += 1
                line.timestamp = self.time
                return True

        self.misses += 1
        self.insert(set_index, tag)
        return False

    def insert(self, set_index, tag):
        lines = self.cache[set_index]
        if len(lines) < self.associativity:
            lines.append(CacheLine(tag))
        else:
            if self.replacement_policy == "LRU":
                evict = min(lines, key=lambda x: x.timestamp)
            elif self.replacement_policy == "Random":
                evict = random.choice(lines)
            elif self.replacement_policy == "FIFO":
                evict = lines[0]
            lines.remove(evict)
            lines.append(CacheLine(tag))
        lines[-1].timestamp = self.time

# -------------------------------
# Multi-Level Cache Simulator
# -------------------------------
class MultiLevelCacheSimulator:
    def __init__(self, l1_params, l2_params):
        self.L1 = CacheSimulator(**l1_params)
        self.L2 = CacheSimulator(**l2_params)
        self.mem_accesses = 0
        self.total_accesses = 0
        self.total_cycles = 0

        self.l1_latency = 1
        self.l2_latency = 5
        self.mem_latency = 50

    def access(self, address):
        self.total_accesses += 1

        if self.L1.access(address):
            self.total_cycles += self.l1_latency
        elif self.L2.access(address):
            set_index, tag = self.L1.get_set_and_tag(address)
            self.L1.insert(set_index, tag)
            self.total_cycles += self.l2_latency
        else:
            set_index_l2, tag_l2 = self.L2.get_set_and_tag(address)
            set_index_l1, tag_l1 = self.L1.get_set_and_tag(address)
            self.L2.insert(set_index_l2, tag_l2)
            self.L1.insert(set_index_l1, tag_l1)
            self.total_cycles += self.mem_latency
            self.mem_accesses += 1

    def run(self, trace):
        for addr in trace:
            self.access(addr)
        return {
            "L1 Hits": self.L1.hits,
            "L1 Misses": self.L1.misses,
            "L2 Hits": self.L2.hits,
            "L2 Misses": self.L2.misses,
            "Main Memory Accesses": self.mem_accesses,
            "Total Accesses": self.total_accesses,
            "Total Cycles": self.total_cycles,
            "L1 Hit Rate (%)": round((self.L1.hits / self.total_accesses) * 100, 2),
            "L2 Hit Rate (%)": round((self.L2.hits / max(self.L1.misses, 1)) * 100, 2)
        }

In [26]:
def load_trace_from_file(filename):
    trace = []
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) >= 2:
                try:
                    addr = int(parts[1], 16)  # Convert hex to int
                    trace.append(addr)
                except:
                    continue
    return trace

In [27]:
file = "trace4.txt"
trace = load_trace_from_file(file)

In [28]:
# -------------------------------
# Configs to Sweep
# -------------------------------
sizes = [512, 1024]
block_sizes = [32, 64]
associativities = [1, 2, 4]
replacement_policies = ["LRU", "Random", "FIFO"]

In [29]:
# -------------------------------
# CSV Setup
# -------------------------------
csv_file = f"cache_comparison_{file.split(".")[0]}.csv"
header = [
    "Mode", "L1 Size", "L2 Size", "Block Size", "Associativity", "Policy",
    "L1 Hits", "L1 Misses", "L2 Hits", "L2 Misses", "Memory Accesses", "Total Accesses",
    "L1 Hit Rate (%)", "L2 Hit Rate (%)", "Total Cycles"
]

with open(csv_file, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)

    for size, block_size, assoc, policy in itertools.product(sizes, block_sizes, associativities, replacement_policies):

        # --- Single-Level Cache ---
        l1_params = {
            "size": size,
            "block_size": block_size,
            "associativity": assoc,
            "replacement_policy": policy
        }

        l1_sim = CacheSimulator(**l1_params)
        total_accesses = 0
        total_cycles = 0
        mem_latency = 50

        for addr in trace:
            total_accesses += 1
            if l1_sim.access(addr):
                total_cycles += 1
            else:
                total_cycles += mem_latency

        writer.writerow([
            "Single",
            size, "-", block_size, assoc, policy,
            l1_sim.hits,
            l1_sim.misses,
            "-", "-", l1_sim.misses, total_accesses,
            round((l1_sim.hits / total_accesses) * 100, 2),
            "-", total_cycles
        ])

        # --- Multi-Level Cache ---
        l2_params = {
            "size": size * 4,
            "block_size": block_size * 2,
            "associativity": min(assoc * 2, 8),
            "replacement_policy": policy
        }

        ml_sim = MultiLevelCacheSimulator(l1_params, l2_params)
        results = ml_sim.run(trace)

        writer.writerow([
            "Multi",
            size, l2_params["size"], block_size, assoc, policy,
            results["L1 Hits"],
            results["L1 Misses"],
            results["L2 Hits"],
            results["L2 Misses"],
            results["Main Memory Accesses"],
            results["Total Accesses"],
            results["L1 Hit Rate (%)"],
            results["L2 Hit Rate (%)"],
            results["Total Cycles"]
        ])

print(f"✅ Results saved to: {csv_file}")


✅ Results saved to: cache_comparison_trace4.csv
